# Branch & Bound And Brute Force Cyclopeptide Sequencing
Step-by-step execution

## Given Experimental Spectrum

In [9]:

spectrum = [0, 113, 128, 186, 241, 299, 314, 427]
parent_mass = max(spectrum)
print("Spectrum:", spectrum)
print("Parent mass:", parent_mass)


Spectrum: [0, 113, 128, 186, 241, 299, 314, 427]
Parent mass: 427


## Amino Acid Mass Table (simplified)

In [10]:

# Standard integer mass table (20 amino acids)

aa_mass_table = {
    'G': 57,
    'A': 71,
    'S': 87,
    'P': 97,
    'V': 99,
    'T': 101,
    'C': 103,
    'I': 113,
    'L': 113,
    'N': 114,
    'D': 115,
    'K': 128,
    'Q': 128,
    'E': 129,
    'M': 131,
    'H': 137,
    'F': 147,
    'R': 156,
    'Y': 163,
    'W': 186
}

# Unique masses from AA table
all_masses = sorted(set(aa_mass_table.values()))

print("All AA masses:", all_masses)

# Extract masses consistent with spectrum
allowed_masses = [m for m in all_masses if m in spectrum]

print("\nExtracted masses from spectrum:")
print(allowed_masses)




All AA masses: [57, 71, 87, 97, 99, 101, 103, 113, 114, 115, 128, 129, 131, 137, 147, 156, 163, 186]

Extracted masses from spectrum:
[113, 128, 186]


## Helper Functions

In [11]:

def cyclic_spectrum(peptide):
    spec = [0]
    n = len(peptide)
    peptide2 = peptide * 2
    for l in range(1, n):
        for i in range(n):
            spec.append(sum(peptide2[i:i+l]))
    spec.append(sum(peptide))
    return sorted(spec)

def linear_spectrum(peptide):
    spec = [0]
    for i in range(len(peptide)):
        s = 0
        for j in range(i, len(peptide)):
            s += peptide[j]
            spec.append(s)
    return sorted(spec)

def consistent(peptide, spectrum):
    lin = linear_spectrum(peptide)
    for m in lin:
        if lin.count(m) > spectrum.count(m):
            return False
    return True


## Brute Force Algorithm (Try all peptides)

In [12]:

from itertools import product

def brute_force():
    solutions = []
    for length in range(1, 6):
        print("\nTrying length:", length)
        for pep in product(amino_acids, repeat=length):
            pep = list(pep)
            mass = sum(pep)
            print("Testing:", pep, "Mass:", mass)
            if mass == parent_mass:
                spec = cyclic_spectrum(pep)
                print("  Cyclic spectrum:", spec)
                if spec == spectrum:
                    print("  MATCH FOUND!")
                    solutions.append(pep)
    return solutions

print("\nBrute force results:", brute_force())



Trying length: 1
Testing: [113] Mass: 113
Testing: [128] Mass: 128
Testing: [186] Mass: 186

Trying length: 2
Testing: [113, 113] Mass: 226
Testing: [113, 128] Mass: 241
Testing: [113, 186] Mass: 299
Testing: [128, 113] Mass: 241
Testing: [128, 128] Mass: 256
Testing: [128, 186] Mass: 314
Testing: [186, 113] Mass: 299
Testing: [186, 128] Mass: 314
Testing: [186, 186] Mass: 372

Trying length: 3
Testing: [113, 113, 113] Mass: 339
Testing: [113, 113, 128] Mass: 354
Testing: [113, 113, 186] Mass: 412
Testing: [113, 128, 113] Mass: 354
Testing: [113, 128, 128] Mass: 369
Testing: [113, 128, 186] Mass: 427
  Cyclic spectrum: [0, 113, 128, 186, 241, 299, 314, 427]
  MATCH FOUND!
Testing: [113, 186, 113] Mass: 412
Testing: [113, 186, 128] Mass: 427
  Cyclic spectrum: [0, 113, 128, 186, 241, 299, 314, 427]
  MATCH FOUND!
Testing: [113, 186, 186] Mass: 485
Testing: [128, 113, 113] Mass: 354
Testing: [128, 113, 128] Mass: 369
Testing: [128, 113, 186] Mass: 427
  Cyclic spectrum: [0, 113, 128, 18

## Branch and Bound Algorithm

In [13]:

def branch_and_bound():
    peptides = [[]]
    final = []

    while peptides:
        new_list = []
        for pep in peptides:
            for a in amino_acids:
                new_pep = pep + [a]
                print("Expand:", new_pep, "Mass:", sum(new_pep))

                if sum(new_pep) == parent_mass:
                    spec = cyclic_spectrum(new_pep)
                    print("  Full peptide spectrum:", spec)
                    if spec == spectrum:
                        print("  VALID PEPTIDE FOUND!")
                        final.append(new_pep)

                elif sum(new_pep) < parent_mass:
                    if consistent(new_pep, spectrum):
                        print("  Kept (consistent)")
                        new_list.append(new_pep)
                    else:
                        print("  Pruned (inconsistent)")

                else:
                    print("  Pruned (mass exceeded)")

        peptides = new_list

    return final

print("\nBranch & Bound results:", branch_and_bound())


Expand: [113] Mass: 113
  Kept (consistent)
Expand: [128] Mass: 128
  Kept (consistent)
Expand: [186] Mass: 186
  Kept (consistent)
Expand: [113, 113] Mass: 226
  Pruned (inconsistent)
Expand: [113, 128] Mass: 241
  Kept (consistent)
Expand: [113, 186] Mass: 299
  Kept (consistent)
Expand: [128, 113] Mass: 241
  Kept (consistent)
Expand: [128, 128] Mass: 256
  Pruned (inconsistent)
Expand: [128, 186] Mass: 314
  Kept (consistent)
Expand: [186, 113] Mass: 299
  Kept (consistent)
Expand: [186, 128] Mass: 314
  Kept (consistent)
Expand: [186, 186] Mass: 372
  Pruned (inconsistent)
Expand: [113, 128, 113] Mass: 354
  Pruned (inconsistent)
Expand: [113, 128, 128] Mass: 369
  Pruned (inconsistent)
Expand: [113, 128, 186] Mass: 427
  Full peptide spectrum: [0, 113, 128, 186, 241, 299, 314, 427]
  VALID PEPTIDE FOUND!
Expand: [113, 186, 113] Mass: 412
  Pruned (inconsistent)
Expand: [113, 186, 128] Mass: 427
  Full peptide spectrum: [0, 113, 128, 186, 241, 299, 314, 427]
  VALID PEPTIDE FOUND!